In [ ]:
!pip install transformers datasets accelerate bitsandbytes -q

In [ ]:
from datasets import load_dataset

# Stream the 12GB CodeParrot Python dataset
dataset = load_dataset("codeparrot/codeparrot-clean-train", streaming=True)
train_dataset = dataset["train"]

print("Streaming dataset loaded!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


Resolving data files:   0%|          | 0/53 [00:00<?, ?it/s]

Streaming dataset loaded!


## DistilGPT2

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    return tokenizer(example["content"], truncation=True, max_length=128)

# Lazy mapping keeps streaming behavior
tokenized_train = train_dataset.map(tokenize_function)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForCausalLM, DataCollatorForLanguageModeling

model = AutoModelForCausalLM.from_pretrained("distilgpt2")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Use a folder on your Drive for outputs
output_dir = "/content/drive/MyDrive/codegen_training/distilgpt2-python-12GB"

Mounted at /content/drive


In [ ]:
from transformers import Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    max_steps=5000,  # specify number of steps (instead of epochs)
    save_steps=1000,
    save_total_limit=3,
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none",
    dataloader_num_workers=2,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()
model.save_pretrained("./syntax_pretrained_model")
tokenizer.save_pretrained("./syntax_pretrained_model")

/tmp/ipython-input-540195222.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.464300
100,3.111900
150,2.874300
200,2.595800
250,2.506000
300,2.366900
350,2.338900
400,2.243600
450,2.159700
500,2.119900


## DeBERTa

In [ ]:
from transformers import AutoTokenizer

# Use DeBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
tokenizer.pad_token = tokenizer.eos_token if tokenizer.eos_token else tokenizer.pad_token

def tokenize_function(example):
    # same as before, you can keep your dataset setup
    return tokenizer(example["content"], truncation=True, max_length=128)

tokenized_train = train_dataset.map(tokenize_function)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Use masked language model (MLM) objective
from transformers import AutoModelForMaskedLM, DataCollatorForLanguageModeling

model = AutoModelForMaskedLM.from_pretrained("microsoft/deberta-v3-base")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# Save outputs to Drive
from google.colab import drive
drive.mount('/content/drive')

output_dir = "/content/drive/MyDrive/codegen_training/deberta-python-12GB"

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Mounted at /content/drive


In [ ]:
# Training setup (same as before)
from transformers import Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    max_steps=5000,
    save_steps=1000,
    save_total_limit=3,
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none",
    dataloader_num_workers=2,
)

# Trainer (no change here)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

# Save model for reuse later
model.save_pretrained("./deberta_syntax_pretrained_model")
tokenizer.save_pretrained("./deberta_syntax_pretrained_model")

/tmp/ipython-input-1118481240.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1, 'pad_token_id': 2}.


Step,Training Loss
50,14.867700
100,10.564500
150,7.761800
200,6.592000
250,5.735600
300,5.070000
350,4.574000
400,4.181100
450,3.757700
500,3.559300


Step,Training Loss
50,14.867700
100,10.564500
150,7.761800
200,6.592000
250,5.735600
300,5.070000
350,4.574000
400,4.181100
450,3.757700
500,3.559300


KeyboardInterrupt: 